<h1>AnimalCLEF25 @ CVPR-FGVC & LifeCLEF<h1>

ImageNet Benchmark:

https://paperswithcode.com/sota/image-classification-on-imagenet?p=deepvit-towards-deeper-vision-transformer

Swin Checkpoints:

https://github.com/microsoft/Swin-Transformer/blob/main/MODELHUB.md#simmim-pretrained-swin-v2-models

# **Data Analysis**

**AnimalCLEF**

In [ ]:
import pandas as pd

df = pd.read_csv("metadata.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15209 entries, 0 to 15208
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_id     15209 non-null  int64 
 1   identity     13074 non-null  object
 2   path         15209 non-null  object
 3   date         11302 non-null  object
 4   orientation  14506 non-null  object
 5   species      13821 non-null  object
 6   split        15209 non-null  object
 7   dataset      15209 non-null  object
dtypes: int64(1), object(7)
memory usage: 950.7+ KB


In [ ]:
database_df = df[df['split'] == 'database']
query_df = df[df['split'] == 'query']

In [ ]:
database_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13074 entries, 0 to 14708
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_id     13074 non-null  int64 
 1   identity     13074 non-null  object
 2   path         13074 non-null  object
 3   date         10113 non-null  object
 4   orientation  12871 non-null  object
 5   species      11686 non-null  object
 6   split        13074 non-null  object
 7   dataset      13074 non-null  object
dtypes: int64(1), object(7)
memory usage: 919.3+ KB


In [ ]:
database_df["identity"].info()

<class 'pandas.core.series.Series'>
Index: 13074 entries, 0 to 14708
Series name: identity
Non-Null Count  Dtype 
--------------  ----- 
13074 non-null  object
dtypes: object(1)
memory usage: 204.3+ KB


In [ ]:
database_df = pd.read_csv('database_metadata.csv')

identity_counts = database_df['identity'].value_counts()
single_occurrence_identities = identity_counts[identity_counts == 1].index

print(single_occurrence_identities)
print()
print(len(single_occurrence_identities))

Index(['SalamanderID2025_250', 'SalamanderID2025_216', 'SalamanderID2025_217',
       'SalamanderID2025_221', 'SalamanderID2025_222', 'SalamanderID2025_223',
       'SalamanderID2025_224', 'SalamanderID2025_225', 'SalamanderID2025_229',
       'SeaTurtleID2022_t103',
       ...
       'SalamanderID2025_126', 'SalamanderID2025_125', 'SalamanderID2025_124',
       'SalamanderID2025_167', 'SalamanderID2025_166', 'SalamanderID2025_163',
       'SalamanderID2025_159', 'SalamanderID2025_158', 'SalamanderID2025_154',
       'SalamanderID2025_153'],
      dtype='object', name='identity', length=317)

317


In [ ]:
query_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2135 entries, 3 to 15208
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_id     2135 non-null   int64 
 1   identity     0 non-null      object
 2   path         2135 non-null   object
 3   date         1189 non-null   object
 4   orientation  1635 non-null   object
 5   species      2135 non-null   object
 6   split        2135 non-null   object
 7   dataset      2135 non-null   object
dtypes: int64(1), object(7)
memory usage: 150.1+ KB


In [ ]:
database_df.to_csv("database_metadata.csv", index=False)
query_df.to_csv("query_metadata.csv", index=False)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Load the database_metadata.csv
database_df = pd.read_csv('database_metadata.csv')

# 2. Find identities that appear only once
identity_counts = database_df['identity'].value_counts()
single_occurrence_identities = identity_counts[identity_counts == 1].index

# 3. Split the data:
# - Images with identities appearing only once
single_occurrence_df = database_df[database_df['identity'].isin(single_occurrence_identities)]
print("single_occurrence_df:")
single_occurrence_df.info()
print()

# - Images with identities appearing at least twice
normal_df = database_df[~database_df['identity'].isin(single_occurrence_identities)]
print("normal_df:")
normal_df.info()
print()

# 4. Perform stratified train/validation split on normal identities
train_normal_df, test_normal_df = train_test_split(
    normal_df,
    test_size=0.2,
    stratify=normal_df['identity'],
    random_state=42
)
print("train_normal.df:")
train_normal_df.info()
print()
print("test_normal.df:")
test_normal_df.info()
print()

# 5. Combine all single-occurrence identities + 80% of normal identities into the training set
train_df = pd.concat([train_normal_df, single_occurrence_df]).reset_index(drop=True)

# 6. Save the CSV files
train_df.to_csv('train_database_metadata.csv', index=False)
test_normal_df.to_csv('test_database_metadata.csv', index=False)

print(f"Train set: {len(train_df)} images")
print(f"Test set: {len(test_normal_df)} images")
split = len(test_normal_df)/len(train_df)
print(f"Split: {split}")

single_occurrence_df:
<class 'pandas.core.frame.DataFrame'>
Index: 317 entries, 678 to 6309
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_id     317 non-null    int64 
 1   identity     317 non-null    object
 2   path         317 non-null    object
 3   date         308 non-null    object
 4   orientation  317 non-null    object
 5   species      7 non-null      object
 6   split        317 non-null    object
 7   dataset      317 non-null    object
dtypes: int64(1), object(7)
memory usage: 22.3+ KB

normal_df:
<class 'pandas.core.frame.DataFrame'>
Index: 12757 entries, 0 to 13073
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_id     12757 non-null  int64 
 1   identity     12757 non-null  object
 2   path         12757 non-null  object
 3   date         9805 non-null   object
 4   orientation  12554 non-null  object
 5   species 

In [ ]:
import pandas as pd

train_df = pd.read_csv('train_database_metadata.csv')
test_df = pd.read_csv('test_database_metadata.csv')

train_identities = set(train_df['identity'].unique())
test_identities = set(test_df['identity'].unique())

missing_identities = test_identities - train_identities

if len(missing_identities) == 0:
    print("✅ All identities in the test set are present in the train set.")
else:
    print(f"❌ {len(missing_identities)} identities in the test set are missing from the train set!")
    print("Missing identities:")
    print(missing_identities)

✅ All identities in the test set are present in the train set.


**WildlifeReID-10k**

In [ ]:
import pandas as pd

csv_path = "/content/metadata.csv"

df = pd.read_csv(csv_path)

print("Split values:", df["split"].unique())

train_df = df[df["split"] == "train"].reset_index(drop=True)
test_df = df[df["split"] == "test"].reset_index(drop=True)

train_df.to_csv("train_metadata.csv", index=False)
test_df.to_csv("test_metadata.csv", index=False)

print(f"✅ Train set: {len(train_df)} samples")
print(f"✅ Test set: {len(test_df)} samples")

<ipython-input-3-a4784bcf0197>:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Split values: ['train' 'test']
✅ Train set: 109927 samples
✅ Test set: 30561 samples


In [ ]:
import pandas as pd

train_df = pd.read_csv("train_metadata.csv")
test_df = pd.read_csv("test_metadata.csv")

train_identities = set(train_df["identity"].unique())
test_identities = set(test_df["identity"].unique())

unseen_identities = test_identities - train_identities

print(f"Number of unseen identities in test: {len(unseen_identities)}")
print("Unseen identities:")
for identity in sorted(unseen_identities):
    print(identity)

Număr de identități necunoscute în test: 523
Identitățile necunoscute:
AAUZebraFish_2
ATRW_118
ATRW_12
ATRW_181
ATRW_226
ATRW_262
ATRW_270
ATRW_49
ATRW_73
AerialCattle2017_10
AerialCattle2017_5
AmvrakikosTurtles_36
AmvrakikosTurtles_40
AmvrakikosTurtles_42
BelugaID_whale022
BelugaID_whale035
BelugaID_whale038
BelugaID_whale067
BelugaID_whale088
BelugaID_whale089
BelugaID_whale180
BelugaID_whale224
BelugaID_whale298
BelugaID_whale306
BelugaID_whale339
BelugaID_whale342
BelugaID_whale346
BelugaID_whale350
BelugaID_whale371
BelugaID_whale381
BelugaID_whale398
BelugaID_whale410
BelugaID_whale437
BelugaID_whale450
BelugaID_whale456
BelugaID_whale474
BelugaID_whale483
BelugaID_whale486
BelugaID_whale490
BelugaID_whale531
BelugaID_whale536
BelugaID_whale543
BelugaID_whale578
BelugaID_whale607
BelugaID_whale620
BelugaID_whale629
BelugaID_whale668
BelugaID_whale669
BelugaID_whale698
BelugaID_whale716
BelugaID_whale717
BelugaID_whale721
BelugaID_whale731
BelugaID_whale733
BelugaID_whale779
BirdI

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Load the database_metadata.csv
database_df = pd.read_csv('metadata.csv')

# 2. Find identities that appear only once
identity_counts = database_df['identity'].value_counts()
single_occurrence_identities = identity_counts[identity_counts == 1].index

# 3. Split the data:
# - Images with identities appearing only once
single_occurrence_df = database_df[database_df['identity'].isin(single_occurrence_identities)]
print("single_occurrence_df:")
single_occurrence_df.info()
print()

# - Images with identities appearing at least twice
normal_df = database_df[~database_df['identity'].isin(single_occurrence_identities)]
print("normal_df:")
normal_df.info()
print()

# 4. Perform stratified train/validation split on normal identities
train_normal_df, test_normal_df = train_test_split(
    normal_df,
    test_size=0.2,
    stratify=normal_df['identity'],
    random_state=42
)
print("train_normal.df:")
train_normal_df.info()
print()
print("test_normal.df:")
test_normal_df.info()
print()

# 5. Combine all single-occurrence identities + 80% of normal identities into the training set
train_df = pd.concat([train_normal_df, single_occurrence_df]).reset_index(drop=True)

# 6. Save the CSV files
train_df.to_csv('train_metadata.csv', index=False)
test_normal_df.to_csv('test_metadata.csv', index=False)

print(f"Train set: {len(train_df)} images")
print(f"Test set: {len(test_normal_df)} images")
split = len(test_normal_df)/len(train_df)
print(f"Split: {split}")

<ipython-input-2-6a2a8bb70a60>:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  database_df = pd.read_csv('metadata.csv')


single_occurrence_df:
<class 'pandas.core.frame.DataFrame'>
Index: 1019 entries, 40128 to 138026
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   identity     1019 non-null   object
 1   path         1019 non-null   object
 2   date         912 non-null    object
 3   orientation  993 non-null    object
 4   species      1019 non-null   object
 5   split        1019 non-null   object
 6   dataset      1019 non-null   object
 7   cluster_id   911 non-null    object
dtypes: object(8)
memory usage: 71.6+ KB

normal_df:
<class 'pandas.core.frame.DataFrame'>
Index: 139469 entries, 0 to 140487
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   identity     139469 non-null  object
 1   path         139469 non-null  object
 2   date         46940 non-null   object
 3   orientation  49524 non-null   object
 4   species      139469 non-null  object
 5   spli

# **Create Dataloaders**

In [ ]:
%%writefile create_dataloaders.py
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os
import torchvision.transforms as T
import torch

class AnimalDataset(Dataset):
    def __init__(self, csv_file, root_dir, label_encoder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_encoder = label_encoder
        self.label_decoder = {v: k for k, v in label_encoder.items()}  # reverse mapping

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['path'])
        identity = self.data.iloc[idx]['identity']

        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.label_encoder[identity]

        return image, label, img_path, identity  # return more info

# Root directory where images are located
root = './animal-clef-2025'

# Simple resize for displaying images
transform_display = T.Compose([
    T.Resize([384, 384]),
])

transform = T.Compose([
    *transform_display.transforms,
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

train_csv = "train_database_metadata.csv"
test_csv = "test_database_metadata.csv"

train_identities = pd.read_csv(train_csv)['identity'].unique()
label_encoder = {identity: idx for idx, identity in enumerate(sorted(train_identities))}

# Datasets
train_dataset = AnimalDataset(
    csv_file=train_csv,
    root_dir=root,
    label_encoder=label_encoder,
    transform=transform
)

test_dataset = AnimalDataset(
    csv_file=test_csv,
    root_dir=root,
    label_encoder=label_encoder,
    transform=transform
)

batch_size = 2

def collate_fn(batch):
    images, labels, paths, identities = zip(*batch)
    images = torch.stack(images)
    labels = torch.tensor(labels)
    return images, labels, paths, identities

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, collate_fn=collate_fn)

print(f"Number of training batches: {len(train_loader)}")
print(f"Number of testing batches: {len(test_loader)}")

images, labels, paths, identities = next(iter(train_loader))

print(f"\nTrain batch - images shape: {images.shape}")
print(f"Train batch - labels shape: {labels.shape}")
print(f"Train batch - labels example: {labels[:5]}")

for i in range(len(paths)):
    print(f"[Train] Image Path: {paths[i]} | Label ID: {labels[i].item()} | Identity: {identities[i]}")

images_test, labels_test, paths_test, identities_test = next(iter(test_loader))

print(f"\nTest batch - images shape: {images_test.shape}")
print(f"Test batch - labels shape: {labels_test.shape}")
print(f"Test batch - labels example: {labels_test[:5]}")

for i in range(len(paths_test)):
    print(f"[Test] Image Path: {paths_test[i]} | Label ID: {labels_test[i].item()} | Identity: {identities_test[i]}")


Overwriting create_dataloaders.py


In [ ]:
%%writefile create_dataloaders.py
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os
import torchvision.transforms as T

class AnimalDataset(Dataset):
    def __init__(self, csv_file, root_dir, label_encoder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Build the full path to the image
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['path'])

        # Open the image
        image = Image.open(img_path).convert('RGB')

        # Apply transformations (if any)
        if self.transform:
            image = self.transform(image)

        # Label is the value from "identity"
        label_str = self.data.iloc[idx]['identity']
        label = self.label_encoder[label_str]

        return image, label

# Root directory where images are located
root = './animal-clef-2025'

# Simple resize for displaying images
transform_display = T.Compose([
    T.Resize([384, 384]),
])

# Transform for training / evaluation (resize + normalization)
transform = T.Compose([
    *transform_display.transforms,   # reuse resize step
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

train_csv = "train_database_metadata.csv"
test_csv = "test_database_metadata.csv"

train_identities = pd.read_csv(train_csv)['identity'].unique()
label_encoder = {identity: idx for idx, identity in enumerate(sorted(train_identities))}

# Datasets
train_dataset = AnimalDataset(
    csv_file=train_csv,
    root_dir=root,
    label_encoder=label_encoder,
    transform=transform
)

test_dataset = AnimalDataset(
    csv_file=test_csv,
    root_dir=root,
    label_encoder=label_encoder,
    transform=transform
)

batch_size = 2

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Number of training batches: {len(train_loader)}")
print(f"Number of testing batches: {len(test_loader)}")

images, labels = next(iter(train_loader))

print(f"\nTrain batch - images shape: {images.shape}")
print(f"Train batch - labels shape: {labels.shape}")
print(f"Train batch - labels example: {labels[:5]}")

images_test, labels_test = next(iter(test_loader))

print(f"\nTest batch - images shape: {images_test.shape}")
print(f"Test batch - labels shape: {labels_test.shape}")
print(f"Test batch - labels example: {labels_test[:5]}")

Writing create_dataloaders.py


# **Fine-Tune**

In [ ]:
%%writefile fine_tune_animal_clef.py
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from transformers import get_scheduler
import timm
import csv
import torch.nn as nn
import torch
import torch.optim as optim
import pandas as pd
import os
import torchvision.transforms as T
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class AnimalDataset(Dataset):
    def __init__(self, csv_file, root_dir, label_encoder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Build the full path to the image
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['path'])

        # Open the image
        image = Image.open(img_path).convert('RGB')

        # Apply transformations (if any)
        if self.transform:
            image = self.transform(image)

        # Label is the value from "identity"
        label_str = self.data.iloc[idx]['identity']
        label = self.label_encoder[label_str]

        return image, label

# Early stopping class
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0

    def should_stop(self, current_loss):
        if current_loss < self.best_loss - self.min_delta:
            self.best_loss = current_loss
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

def train_one_epoch(model, dataloader, optimizer, scheduler, criterion, device, epoch):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(dataloader, desc=f"Training (Epoch {epoch+1})"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()

    return running_loss / len(dataloader)

def validate(model, dataloader, criterion, device, epoch):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc=f"Validating (Epoch {epoch+1})"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return running_loss / len(dataloader), accuracy

# Root directory where images are located
root = './animal-clef-2025'

# Transforms for train and validation
train_transform = T.Compose([
    T.RandomResizedCrop(384, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=10),
    T.ColorJitter(0.4, 0.4, 0.4, 0.1),
    T.AutoAugment(policy=T.AutoAugmentPolicy.IMAGENET),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    T.RandomErasing(p=0.25, scale=(0.02, 0.33))
])

test_transform = T.Compose([
    T.Resize([384, 384]),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

train_csv = "train_database_metadata.csv"
test_csv = "test_database_metadata.csv"

train_identities = pd.read_csv(train_csv)['identity'].unique()
label_encoder = {identity: idx for idx, identity in enumerate(sorted(train_identities))}

# Datasets
train_dataset = AnimalDataset(
    csv_file=train_csv,
    root_dir=root,
    label_encoder=label_encoder,
    transform=train_transform
)

test_dataset = AnimalDataset(
    csv_file=test_csv,
    root_dir=root,
    label_encoder=label_encoder,
    transform=test_transform
)

batch_size = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

model = timm.create_model(
    #'vit_large_patch16_384',
    'convnext_xlarge.fb_in22k_ft_in1k_384',
    #'swin_large_patch4_window12_384',
    pretrained=True,
    num_classes=len(label_encoder)
)
model = model.to(device)

num_epochs = 30
lr = 8e-5
weight_decay = 0.01

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

num_training_steps = len(train_loader) * num_epochs
num_warmup_steps = int(0.1 * num_training_steps)

scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

metrics_csv = "epoch_metrics.csv"

if not os.path.exists(metrics_csv):
    with open(metrics_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["epoch", "train_loss", "val_loss", "val_acc", "learning_rate"])

train_losses = []
val_losses = []
val_accuracies = []
learning_rates = []

best_val_loss = float('inf')
best_val_acc = 0.0
best_model_path = "best_model.pth"

early_stopping = EarlyStopping(patience=8, min_delta=0.0005)

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, scheduler, criterion, device, epoch)
    val_loss, val_acc = validate(model, test_loader, criterion, device, epoch)

    current_lr = scheduler.get_last_lr()[0]
    learning_rates.append(current_lr)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | "
          f"Val Acc: {val_acc:.4f} | "
          f"LR: {current_lr:.8f}")

    with open(metrics_csv, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([epoch+1, train_loss, val_loss, val_acc, current_lr])

    # Save best model if val_loss decreased and val_acc increased
    if val_loss < best_val_loss and val_acc > best_val_acc:
        best_val_loss = val_loss
        best_val_acc = val_acc
        torch.save({
            'model_state_dict': model.state_dict(),
            'label_encoder': label_encoder,
        }, best_model_path)
        print(f"✅ Best model saved at epoch {epoch+1} with Val Loss {val_loss:.4f} and Val Acc {val_acc:.4f}")

    # Early stopping check
    if early_stopping.should_stop(val_loss):
        print("🛑 Early stopping triggered!")
        break

# Plot Train and Validation Loss
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.legend()
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid()
plt.savefig("loss_plot.png")
plt.close()

# Plot Validation Accuracy
plt.plot(val_accuracies, label="Validation Accuracy")
plt.legend()
plt.title("Validation Accuracy Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid()
plt.savefig("accuracy_plot.png")
plt.close()

# Plot Learning Rate over Epochs
plt.plot(learning_rates, label="Learning Rate")
plt.legend()
plt.title("Learning Rate Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")
plt.grid()
plt.savefig("learning_rate_plot.png")
plt.close()

Overwriting fine_tune_animal_clef.py


In [ ]:
%%writefile fine_tune_animal_clef.py
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from transformers import get_scheduler
import timm
import csv
import torch.nn as nn
import torch
import torch.optim as optim
import pandas as pd
import os
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class AnimalDataset(Dataset):
    def __init__(self, csv_file, root_dir, label_encoder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Build the full path to the image
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['path'])

        # Open the image
        image = Image.open(img_path).convert('RGB')

        # Apply transformations (if any)
        if self.transform:
            image = self.transform(image)

        # Label is the value from "identity"
        label_str = self.data.iloc[idx]['identity']
        label = self.label_encoder[label_str]

        return image, label

# Early stopping class
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0

    def should_stop(self, current_loss):
        if current_loss < self.best_loss - self.min_delta:
            self.best_loss = current_loss
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

def train_one_epoch(model, dataloader, optimizer, scheduler, criterion, device, epoch):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(dataloader, desc=f"Training (Epoch {epoch+1})"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()

    return running_loss / len(dataloader)

def validate(model, dataloader, criterion, device, epoch):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc=f"Validating (Epoch {epoch+1})"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return running_loss / len(dataloader), accuracy

class GLUBlock(nn.Module):
    def __init__(self, dim_in, dim_out, dropout=0.3):
        super().__init__()
        self.linear = nn.Linear(dim_in, dim_out * 2)
        self.norm = nn.LayerNorm(dim_out)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x_proj = self.linear(x)
        out, gate = x_proj.chunk(2, dim=-1)
        return self.norm(out * torch.sigmoid(gate))

class MultiBackboneClassifier(nn.Module):
    def __init__(self, model_names, embedding_dims, num_classes):
        super().__init__()
        assert len(model_names) == len(embedding_dims)

        self.backbones = nn.ModuleList([
            timm.create_model(name, pretrained=True, num_classes=0)
            for name in model_names
        ])

        total_dim = sum(embedding_dims)

        total_dim = sum(embedding_dims)

        self.classifier = nn.Sequential(
            nn.LayerNorm(total_dim),
            GLUBlock(total_dim, 2048, dropout=0.4),
            GLUBlock(2048, 1024, dropout=0.3),
            GLUBlock(1024, 512, dropout=0.2),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        feats = [F.normalize(backbone(x), p=2, dim=1) for backbone in self.backbones]
        fused = torch.cat(feats, dim=1)
        return self.classifier(fused)

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing

    def forward(self, x, target):
        log_probs = torch.nn.functional.log_softmax(x, dim=-1)
        nll_loss = -log_probs.gather(dim=-1, index=target.unsqueeze(1)).squeeze(1)
        smooth_loss = -log_probs.mean(dim=-1)
        return ((1.0 - self.smoothing) * nll_loss + self.smoothing * smooth_loss).mean()

def freeze_backbones(model):
    for backbone in model.backbones:
        for param in backbone.parameters():
            param.requires_grad = False

def unfreeze_backbones(model):
    for backbone in model.backbones:
        for param in backbone.parameters():
            param.requires_grad = True


# Root directory where images are located
root = './animal-clef-2025'

# Transforms for train and validation
train_transform = T.Compose([
    T.RandomResizedCrop(384, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=10),
    T.ColorJitter(0.4, 0.4, 0.4, 0.1),
    T.AutoAugment(policy=T.AutoAugmentPolicy.IMAGENET),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    T.RandomErasing(p=0.25, scale=(0.02, 0.33))
])

test_transform = T.Compose([
    T.Resize([384, 384]),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

train_csv = "train_database_metadata.csv"
test_csv = "test_database_metadata.csv"

train_identities = pd.read_csv(train_csv)['identity'].unique()
label_encoder = {identity: idx for idx, identity in enumerate(sorted(train_identities))}

# Datasets
train_dataset = AnimalDataset(
    csv_file=train_csv,
    root_dir=root,
    label_encoder=label_encoder,
    transform=train_transform
)

test_dataset = AnimalDataset(
    csv_file=test_csv,
    root_dir=root,
    label_encoder=label_encoder,
    transform=test_transform
)

batch_size = 2

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

model_names = [
    'convnext_xlarge.fb_in22k_ft_in1k_384',   # 2048
    'swin_large_patch4_window12_384',         # 1536
    'maxvit_xlarge_tf_384.in21k_ft_in1k'      # 2048
]

embedding_dims = [2048, 1536, 2048]

model = MultiBackboneClassifier(
    model_names=model_names,
    embedding_dims=embedding_dims,
    num_classes=len(label_encoder)
).to(device)

num_epochs = 30

optimizer = optim.AdamW(model.parameters(), lr=8e-5, weight_decay=0.01)
criterion = LabelSmoothingCrossEntropy(smoothing=0.1)

num_training_steps = len(train_loader) * num_epochs
num_warmup_steps = int(0.1 * num_training_steps)

scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

metrics_csv = "epoch_metrics.csv"

if not os.path.exists(metrics_csv):
    with open(metrics_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["epoch", "train_loss", "val_loss", "val_acc", "learning_rate"])

train_losses = []
val_losses = []
val_accuracies = []
learning_rates = []

best_val_loss = float('inf')
best_val_acc = 0.0
best_model_path = "best_model.pth"

early_stopping = EarlyStopping(patience=8, min_delta=0.0005)

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, scheduler, criterion, device, epoch)
    val_loss, val_acc = validate(model, test_loader, criterion, device, epoch)

    current_lr = scheduler.get_last_lr()[0]
    learning_rates.append(current_lr)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | "
          f"Val Acc: {val_acc:.4f} | "
          f"LR: {current_lr:.8f}")

    with open(metrics_csv, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([epoch+1, train_loss, val_loss, val_acc, current_lr])

    # Save best model if val_loss decreased and val_acc increased
    if val_loss < best_val_loss and val_acc > best_val_acc:
        best_val_loss = val_loss
        best_val_acc = val_acc
        torch.save({
            'model_state_dict': model.state_dict(),
            'label_encoder': label_encoder,
        }, best_model_path)
        print(f"✅ Best model saved at epoch {epoch+1} with Val Loss {val_loss:.4f} and Val Acc {val_acc:.4f}")

    # Early stopping check
    if early_stopping.should_stop(val_loss):
        print("🛑 Early stopping triggered!")
        break

# Plot Train and Validation Loss
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.legend()
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid()
plt.savefig("loss_plot.png")
plt.close()

# Plot Validation Accuracy
plt.plot(val_accuracies, label="Validation Accuracy")
plt.legend()
plt.title("Validation Accuracy Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid()
plt.savefig("accuracy_plot.png")
plt.close()

# Plot Learning Rate over Epochs
plt.plot(learning_rates, label="Learning Rate")
plt.legend()
plt.title("Learning Rate Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")
plt.grid()
plt.savefig("learning_rate_plot.png")
plt.close()

Writing fine_tune_animal_clef.py


In [ ]:
%%writefile fine_tune_animal_clef.py
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from transformers import get_scheduler
import timm
import csv
import torch.nn as nn
import torch
import torch.optim as optim
import pandas as pd
import os
import torchvision.transforms as T
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class AnimalDataset(Dataset):
    def __init__(self, csv_file, root_dir, label_encoder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Build the full path to the image
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['path'])

        # Open the image
        image = Image.open(img_path).convert('RGB')

        # Apply transformations (if any)
        if self.transform:
            image = self.transform(image)

        # Label is the value from "identity"
        label_str = self.data.iloc[idx]['identity']
        label = self.label_encoder[label_str]

        return image, label

# Early stopping class
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0

    def should_stop(self, current_loss):
        if current_loss < self.best_loss - self.min_delta:
            self.best_loss = current_loss
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

class EnsembleNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone1 = timm.create_model(
            'swin_large_patch4_window12_384',
            pretrained=True,
            num_classes=0
        )
        self.backbone2 = timm.create_model(
            'convnext_xlarge.fb_in22k_ft_in1k_384',
            pretrained=True,
            num_classes=0
        )
        self.backbone3 = timm.create_model(
            'beit_large_patch16_384.in22k_ft_in22k_in1k',
            pretrained=True,
            num_classes=0
        )

        self.feat_dim1 = self.backbone1.num_features
        self.feat_dim2 = self.backbone2.num_features
        self.feat_dim3 = self.backbone3.num_features

        self.classifier = nn.Sequential(
            nn.LayerNorm(self.feat_dim1 + self.feat_dim2 + self.feat_dim3),
            nn.Linear(self.feat_dim1 + self.feat_dim2 + self.feat_dim3, 4096),
            nn.GELU(),
            nn.Dropout(0.4),
            nn.Linear(4096, 2048),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(2048, num_classes)
        )

    def forward(self, x):
        f1 = self.backbone1(x)
        f2 = self.backbone2(x)
        f3 = self.backbone3(x)
        fused = torch.cat([f1, f2, f3], dim=1)
        return self.classifier(fused)

def train_one_epoch(model, dataloader, optimizer, scheduler, criterion, device, epoch):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(dataloader, desc=f"Training (Epoch {epoch+1})"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()

    return running_loss / len(dataloader)

def validate(model, dataloader, criterion, device, epoch):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc=f"Validating (Epoch {epoch+1})"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return running_loss / len(dataloader), accuracy

# Root directory where images are located
root = './animal-clef-2025'

# Transforms for train and validation
train_transform = T.Compose([
    T.RandomResizedCrop(384, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=10),
    T.ColorJitter(0.4, 0.4, 0.4, 0.1),
    T.AutoAugment(policy=T.AutoAugmentPolicy.IMAGENET),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    T.RandomErasing(p=0.25, scale=(0.02, 0.33))
])

test_transform = T.Compose([
    T.Resize([384, 384]),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

train_csv = "train_database_metadata.csv"
test_csv = "test_database_metadata.csv"

train_identities = pd.read_csv(train_csv)['identity'].unique()
label_encoder = {identity: idx for idx, identity in enumerate(sorted(train_identities))}

# Datasets
train_dataset = AnimalDataset(
    csv_file=train_csv,
    root_dir=root,
    label_encoder=label_encoder,
    transform=train_transform
)

test_dataset = AnimalDataset(
    csv_file=test_csv,
    root_dir=root,
    label_encoder=label_encoder,
    transform=test_transform
)

batch_size = 2

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

model = EnsembleNet(num_classes=len(label_encoder)).to(device)

model = model.to(device)

num_epochs = 30
lr = 2e-5
weight_decay = 0.01

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

num_training_steps = len(train_loader) * num_epochs
num_warmup_steps = int(0.1 * num_training_steps)

scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

metrics_csv = "epoch_metrics.csv"

if not os.path.exists(metrics_csv):
    with open(metrics_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["epoch", "train_loss", "val_loss", "val_acc", "learning_rate"])

train_losses = []
val_losses = []
val_accuracies = []
learning_rates = []

best_val_loss = float('inf')
best_val_acc = 0.0
best_model_path = "best_model.pth"

early_stopping = EarlyStopping(patience=8, min_delta=0.0005)

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, scheduler, criterion, device, epoch)
    val_loss, val_acc = validate(model, test_loader, criterion, device, epoch)

    current_lr = scheduler.get_last_lr()[0]
    learning_rates.append(current_lr)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | "
          f"Val Acc: {val_acc:.4f} | "
          f"LR: {current_lr:.8f}")

    with open(metrics_csv, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([epoch+1, train_loss, val_loss, val_acc, current_lr])

    # Save best model if val_loss decreased and val_acc increased
    if val_loss < best_val_loss and val_acc > best_val_acc:
        best_val_loss = val_loss
        best_val_acc = val_acc
        torch.save({
            'model_state_dict': model.state_dict(),
            'label_encoder': label_encoder,
        }, best_model_path)
        print(f"✅ Best model saved at epoch {epoch+1} with Val Loss {val_loss:.4f} and Val Acc {val_acc:.4f}")

    # Early stopping check
    if early_stopping.should_stop(val_loss):
        print("🛑 Early stopping triggered!")
        break

# Plot Train and Validation Loss
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.legend()
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid()
plt.savefig("loss_plot.png")
plt.close()

# Plot Validation Accuracy
plt.plot(val_accuracies, label="Validation Accuracy")
plt.legend()
plt.title("Validation Accuracy Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid()
plt.savefig("accuracy_plot.png")
plt.close()

# Plot Learning Rate over Epochs
plt.plot(learning_rates, label="Learning Rate")
plt.legend()
plt.title("Learning Rate Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")
plt.grid()
plt.savefig("learning_rate_plot.png")
plt.close()


Writing fine_tune_animal_clef.py


# **Extract Embeedings**

Database

In [ ]:
%%writefile extract_embeddings.py
import torch
import timm
import numpy as np
from tqdm import tqdm
import pandas as pd
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Dataset ---
class AnimalDataset(Dataset):
    def __init__(self, csv_file, root_dir, label_encoder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['path'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.label_encoder[self.data.iloc[idx]['identity']]
        return image, label

# --- Transform (same as test) ---
transform = T.Compose([
    T.Resize([384, 384]),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

# --- Paths ---
root = './animal-clef-2025'
train_csv = "train_database_metadata.csv"
test_csv = "test_database_metadata.csv"

# --- Label encoder ---
train_identities = pd.read_csv(train_csv)['identity'].unique()
label_encoder = {identity: idx for idx, identity in enumerate(sorted(train_identities))}

# --- Datasets ---
train_dataset = AnimalDataset(train_csv, root, label_encoder, transform)
test_dataset = AnimalDataset(test_csv, root, label_encoder, transform)

# --- Loaders ---
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# --- Load trained model ---
# ConvNeXt model
convnext = timm.create_model(
    'convnext_xlarge.fb_in22k_ft_in1k_384',
    pretrained=False,
    num_classes=len(label_encoder)
)
convnext_ckpt = torch.load("best_model_convnext.pth", map_location=device)
convnext.load_state_dict(convnext_ckpt['model_state_dict'])
convnext = convnext.to(device)
convnext.eval()

# ConvNeXt model v2
convnext_v2 = timm.create_model(
    'convnext_xlarge.fb_in22k_ft_in1k_384',
    pretrained=False,
    num_classes=len(label_encoder)
)
convnext_ckpt_v2 = torch.load("best_model_convnext_2.pth", map_location=device)
convnext_v2.load_state_dict(convnext_ckpt_v2['model_state_dict'])
convnext_v2 = convnext_v2.to(device)
convnext_v2.eval()

# MaxViT model
maxvit = timm.create_model(
    'maxvit_xlarge_tf_384.in21k_ft_in1k',
    pretrained=False,
    num_classes=len(label_encoder)
)
maxvit_ckpt = torch.load("best_model_maxvit.pth", map_location=device)
maxvit.load_state_dict(maxvit_ckpt['model_state_dict'])
maxvit = maxvit.to(device)
maxvit.eval()

def extract_embeddings(dataloader, model, model_name="convnext", split_name="train"):
    all_feats, all_labels = [], []
    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc=f"[{model_name}] Extracting {split_name} embeddings"):
            images = images.to(device)
            features = model.forward_features(images)  # Output: (B, C, H, W) or (B, D)
            features = features.mean(dim=(-1, -2)) if features.dim() == 4 else features  # Global average pool if needed
            all_feats.append(features)
            all_labels.append(labels)

    X = torch.cat(all_feats, dim=0).cpu().numpy()
    y = torch.cat(all_labels, dim=0).cpu().numpy()

    np.savez(f"{split_name}_embeddings_{model_name}.npz", X=X, y=y)
    print(f"✅ Saved {split_name}_embeddings_{model_name}.npz with shape {X.shape}")

# --- Run extraction ---
# Extract embeddings with ConvNeXt
extract_embeddings(train_loader, convnext, "convnext", "train")
extract_embeddings(test_loader, convnext, "convnext", "test")

# Extract embeddings with ConvNeXt v2
extract_embeddings(train_loader, convnext_v2, "convnext_v2", "train")
extract_embeddings(test_loader, convnext_v2, "convnext_v2", "test")

# Extract embeddings with MaxViT
extract_embeddings(train_loader, maxvit, "maxvit", "train")
extract_embeddings(test_loader, maxvit, "maxvit", "test")

Writing extract_embeddings.py


Query

In [ ]:
%%writefile extract_embeddings.py
import torch
import timm
import numpy as np
import pandas as pd
from tqdm import tqdm
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Dataset fără identity ---
class QueryDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['path'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.data.iloc[idx]['image_id']

# --- Transform (same as test) ---
transform = T.Compose([
    T.Resize([384, 384]),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

# --- Paths ---
root = './animal-clef-2025'
query_csv = "query_metadata.csv"
batch_size = 8

# --- Query dataset ---
query_dataset = QueryDataset(query_csv, root, transform)
query_loader = DataLoader(query_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# --- Load label encoder from training set ---
train_csv = "train_database_metadata.csv"
train_identities = pd.read_csv(train_csv)['identity'].unique()
label_encoder = {identity: idx for idx, identity in enumerate(sorted(train_identities))}

# --- Load trained models ---
# ConvNeXt model
convnext = timm.create_model(
    'convnext_xlarge.fb_in22k_ft_in1k_384',
    pretrained=False,
    num_classes=len(label_encoder)
)
convnext_ckpt = torch.load("best_model_convnext.pth", map_location=device)
convnext.load_state_dict(convnext_ckpt['model_state_dict'])
convnext = convnext.to(device)
convnext.eval()

# ConvNeXt model v2
convnext_v2 = timm.create_model(
    'convnext_xlarge.fb_in22k_ft_in1k_384',
    pretrained=False,
    num_classes=len(label_encoder)
)
convnext_ckpt_v2 = torch.load("best_model_convnext_2.pth", map_location=device)
convnext_v2.load_state_dict(convnext_ckpt_v2['model_state_dict'])
convnext_v2 = convnext_v2.to(device)
convnext_v2.eval()

# MaxViT model
maxvit = timm.create_model(
    'maxvit_xlarge_tf_384.in21k_ft_in1k',
    pretrained=False,
    num_classes=len(label_encoder)
)
maxvit_ckpt = torch.load("best_model_maxvit.pth", map_location=device)
maxvit.load_state_dict(maxvit_ckpt['model_state_dict'])
maxvit = maxvit.to(device)
maxvit.eval()

# --- Embedding extractor fără etichete ---
def extract_query_embeddings(dataloader, model, model_name="model"):
    all_feats = []
    image_ids = []
    model.eval()
    with torch.no_grad():
        for images, ids in tqdm(dataloader, desc=f"[{model_name}] Extracting query embeddings"):
            images = images.to(device)
            features = model.forward_features(images)
            features = features.mean(dim=(-1, -2)) if features.dim() == 4 else features
            all_feats.append(features.cpu())
            image_ids.extend(ids)

    X = torch.cat(all_feats, dim=0).numpy()
    image_ids = np.array(image_ids)

    np.savez(f"query_embeddings_{model_name}.npz", X=X, image_ids=image_ids)
    print(f"✅ Saved query_embeddings_{model_name}.npz with shape {X.shape}")

# --- Run extraction ---
extract_query_embeddings(query_loader, convnext, "convnext")
extract_query_embeddings(query_loader, convnext_v2, "convnext_v2")
extract_query_embeddings(query_loader, maxvit, "maxvit")

Overwriting extract_embeddings.py


Database

In [ ]:
%%writefile extract_embeddings.py
import torch
import timm
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Dataset definition ---
class AnimalDataset(Dataset):
    def __init__(self, csv_file, root_dir, label_encoder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['path'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.label_encoder[self.data.iloc[idx]['identity']]
        return image, label

# --- Image transform ---
transform = T.Compose([
    T.Resize((384, 384)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

# --- Paths and config ---
root_dir = './animal-clef-2025'
train_csv = "train_database_metadata.csv"
test_csv = "test_database_metadata.csv"
batch_size = 8

# --- Label encoder ---
train_identities = pd.read_csv(train_csv)['identity'].unique()
label_encoder = {identity: idx for idx, identity in enumerate(sorted(train_identities))}

# --- Datasets and loaders ---
train_dataset = AnimalDataset(train_csv, root_dir, label_encoder, transform)
test_dataset = AnimalDataset(test_csv, root_dir, label_encoder, transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# --- Load Swin transformer model ---
def load_swin_model(ckpt_path, model_name):
    model = timm.create_model(model_name, pretrained=False, num_classes=0)
    checkpoint = torch.load(ckpt_path, map_location=device)
    state_dict = checkpoint['model_state_dict']
    filtered_state = {k: v for k, v in state_dict.items() if not k.startswith('head') and 'fc' not in k}
    model.load_state_dict(filtered_state, strict=False)
    return model.to(device).eval()

swin = load_swin_model("best_model_swin_1.pth", "swin_large_patch4_window12_384")
swin_v2 = load_swin_model("best_model_swin_2.pth", "swin_large_patch4_window12_384")

def extract_embeddings(dataloader, model, model_name, split_name):
    all_features = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc=f"[{model_name}] Extracting {split_name} embeddings (GAP)"):
            images = images.to(device)
            features = model.forward_features(images)

            # Correct GAP: if output is (B, H, W, C), apply mean over dim 1 and 2
            if features.dim() == 4:
                if features.shape[1] == features.shape[2]:  # likely (B, H, W, C)
                    features = features.mean(dim=(1, 2))  # --> (B, C)
                else:  # fallback in case it's (B, C, H, W)
                    features = features.mean(dim=(-1, -2))  # --> (B, C)

            all_features.append(features.cpu())
            all_labels.append(labels)

    X = torch.cat(all_features, dim=0).numpy()
    y = torch.cat(all_labels, dim=0).numpy()

    np.savez(f"{split_name}_embeddings_{model_name}.npz", X=X, y=y)
    print(f"✅ Saved: {split_name}_embeddings_{model_name}.npz | Shape: {X.shape}")

# --- Run extraction ---
extract_embeddings(train_loader, swin, "swin", "train")
extract_embeddings(test_loader, swin, "swin", "test")

extract_embeddings(train_loader, swin_v2, "swin_v2", "train")
extract_embeddings(test_loader, swin_v2, "swin_v2", "test")

Overwriting extract_embeddings.py


Query

In [ ]:
%%writefile extract_embeddings.py
import torch
import timm
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Dataset fără identity --- (folosește image_id)
class QueryDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['path'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.data.iloc[idx]['image_id']

# --- Transform ---
transform = T.Compose([
    T.Resize((384, 384)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

# --- Paths and config ---
root_dir = './animal-clef-2025'
query_csv = "query_metadata.csv"
batch_size = 8

# --- Query loader ---
query_dataset = QueryDataset(query_csv, root_dir, transform)
query_loader = DataLoader(query_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# --- Load Swin transformer model ---
def load_swin_model(ckpt_path, model_name):
    model = timm.create_model(model_name, pretrained=False, num_classes=0)
    checkpoint = torch.load(ckpt_path, map_location=device)
    state_dict = checkpoint['model_state_dict']
    filtered_state = {k: v for k, v in state_dict.items() if not k.startswith('head') and 'fc' not in k}
    model.load_state_dict(filtered_state, strict=False)
    return model.to(device).eval()

swin = load_swin_model("best_model_swin_1.pth", "swin_large_patch4_window12_384")
swin_v2 = load_swin_model("best_model_swin_2.pth", "swin_large_patch4_window12_384")

# --- Extract query embeddings ---
def extract_query_embeddings(dataloader, model, model_name):
    all_features = []
    all_ids = []

    with torch.no_grad():
        for images, image_ids in tqdm(dataloader, desc=f"[{model_name}] Extracting query embeddings (GAP)"):
            images = images.to(device)
            features = model.forward_features(images)

            # Correct GAP: if output is (B, H, W, C), apply mean over dim 1 and 2
            if features.dim() == 4:
                if features.shape[1] == features.shape[2]:  # likely (B, H, W, C)
                    features = features.mean(dim=(1, 2))  # --> (B, C)
                else:  # fallback in case it's (B, C, H, W)
                    features = features.mean(dim=(-1, -2))  # --> (B, C)

            all_features.append(features.cpu())
            all_ids.extend(image_ids)

    X = torch.cat(all_features, dim=0).numpy()
    ids = np.array(all_ids)

    np.savez(f"query_embeddings_{model_name}.npz", X=X, image_ids=ids)
    print(f"✅ Saved: query_embeddings_{model_name}.npz | Shape: {X.shape}")

# --- Run extraction ---
extract_query_embeddings(query_loader, swin, "swin")
extract_query_embeddings(query_loader, swin_v2, "swin_v2")

Overwriting extract_embeddings.py


Database

In [ ]:
%%writefile extract_embeddings.py
import torch
import timm
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Dataset Definition ---
class AnimalDataset(Dataset):
    def __init__(self, csv_file, root_dir, label_encoder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['path'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.label_encoder[self.data.iloc[idx]['identity']]
        return image, label

# --- Transform ---
transform = T.Compose([
    T.Resize((384, 384)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

# --- Paths & Config ---
root_dir = './animal-clef-2025'
train_csv = "train_database_metadata.csv"
test_csv = "test_database_metadata.csv"
batch_size = 8

# --- Label Encoder ---
train_identities = pd.read_csv(train_csv)['identity'].unique()
label_encoder = {identity: idx for idx, identity in enumerate(sorted(train_identities))}

# --- Data Loaders ---
train_dataset = AnimalDataset(train_csv, root_dir, label_encoder, transform)
test_dataset = AnimalDataset(test_csv, root_dir, label_encoder, transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# --- Load MegaDescriptor Model ---
def load_mega_descriptor():
    model = timm.create_model(
        'hf-hub:BVRA/MegaDescriptor-L-384',
        pretrained=True,
        num_classes=0
    )
    model = model.to(device)
    model.eval()
    return model

model = load_mega_descriptor()

# --- Embedding Extraction ---
def extract_embeddings(dataloader, model, model_name, split_name):
    all_features = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc=f"[{model_name}] Extracting {split_name} embeddings"):
            images = images.to(device)
            features = model.forward_features(images)

            # Apply Global Average Pooling if needed
            if features.dim() == 4:
                if features.shape[1] == features.shape[2]:
                    features = features.mean(dim=(1, 2))  # (B, H, W, C) -> (B, C)
                else:
                    features = features.mean(dim=(-1, -2))  # (B, C, H, W) -> (B, C)

            all_features.append(features.cpu())
            all_labels.append(labels)

    X = torch.cat(all_features, dim=0).numpy()
    y = torch.cat(all_labels, dim=0).numpy()
    np.savez(f"{split_name}_embeddings_{model_name}.npz", X=X, y=y)
    print(f"✅ Saved: {split_name}_embeddings_{model_name}.npz | Shape: {X.shape}")

# --- Run Extraction ---
extract_embeddings(train_loader, model, "megadescriptor", "train")
extract_embeddings(test_loader, model, "megadescriptor", "test")

Overwriting extract_embeddings.py


Query

In [ ]:
%%writefile extract_embeddings.py
import torch
import timm
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Dataset pentru query fără etichete ---
class QueryDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['path'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.data.iloc[idx]['image_id']

# --- Transform ---
transform = T.Compose([
    T.Resize((384, 384)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

# --- Config ---
root_dir = './animal-clef-2025'
query_csv = "query_metadata.csv"
batch_size = 8

# --- Query loader ---
query_dataset = QueryDataset(query_csv, root_dir, transform)
query_loader = DataLoader(query_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# --- Load MegaDescriptor ---
def load_mega_descriptor():
    model = timm.create_model(
        'hf-hub:BVRA/MegaDescriptor-L-384',
        pretrained=True,
        num_classes=0
    )
    return model.to(device).eval()

model = load_mega_descriptor()

# --- Extract embeddings for query ---
def extract_query_embeddings(dataloader, model, model_name="megadescriptor"):
    all_features = []
    all_ids = []

    with torch.no_grad():
        for images, image_ids in tqdm(dataloader, desc=f"[{model_name}] Extracting query embeddings"):
            images = images.to(device)
            features = model.forward_features(images)

            # Apply Global Average Pooling if needed
            if features.dim() == 4:
                if features.shape[1] == features.shape[2]:
                    features = features.mean(dim=(1, 2))  # (B, H, W, C) -> (B, C)
                else:
                    features = features.mean(dim=(-1, -2))  # (B, C, H, W) -> (B, C)

            all_features.append(features.cpu())
            all_ids.extend(image_ids)

    X = torch.cat(all_features, dim=0).numpy()
    image_ids = np.array(all_ids)
    np.savez(f"query_embeddings_{model_name}.npz", X=X, image_ids=image_ids)
    print(f"✅ Saved: query_embeddings_{model_name}.npz | Shape: {X.shape}")

# --- Run ---
extract_query_embeddings(query_loader, model, "megadescriptor")

Overwriting extract_embeddings.py


# **Train a Classifier**

In [ ]:
%%writefile mlp_classifier.py
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report
import joblib
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

# --- Define label_encoder consistent with extract_embeddings.py ---
train_csv = "train_database_metadata.csv"
train_identities = pd.read_csv(train_csv)['identity'].unique()
label_encoder = {identity: idx for idx, identity in enumerate(sorted(train_identities))}
idx_to_label = {v: k for k, v in label_encoder.items()}

# --- Load all embeddings ---
def load_embeddings(keys, prefix="train"):
    return {
        key: np.load(f"{prefix}_embeddings_{key}.npz") for key in keys
    }

def pool_if_needed(X):
    return X.mean(axis=(2, 3)) if X.ndim == 4 else X

# --- Scheduler helper ---
def linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps)))
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# --- Keys and data loading ---
keys = ["convnext", "convnext_v2", "maxvit", "swin", "swin_v2", "megadescriptor"]
train_embeddings = load_embeddings(keys, "train")
test_embeddings = load_embeddings(keys, "test")

# --- Preprocess ---
X_train_list, X_test_list = [], []
for k in keys:
    print(f"📦 Using {k}")
    X_train_list.append(pool_if_needed(train_embeddings[k]["X"]))
    X_test_list.append(pool_if_needed(test_embeddings[k]["X"]))

X_train = np.concatenate(X_train_list, axis=1)
X_test = np.concatenate(X_test_list, axis=1)
y_train = train_embeddings[keys[0]]["y"]
y_test = test_embeddings[keys[0]]["y"]

# --- Normalize ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
joblib.dump(scaler, "mlp_scaler.joblib")

# --- PCA ---
pca = PCA(n_components=0.99, svd_solver='full')
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
joblib.dump(pca, "mlp_pca.joblib")

print(f"🔢 Final input dimension after PCA: {X_train_pca.shape[1]}")

# --- Torch datasets ---
X_train_tensor = torch.tensor(X_train_pca, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_pca, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256)

# --- Model ---
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 4096),
            nn.BatchNorm1d(4096),
            nn.SiLU(),
            nn.Dropout(0.3),

            nn.Linear(4096, 2048),
            nn.BatchNorm1d(2048),
            nn.SiLU(),
            nn.Dropout(0.3),

            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.SiLU(),
            nn.Dropout(0.25),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.SiLU(),
            nn.Dropout(0.2),

            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.net(x)

model = MLP(X_train_pca.shape[1], len(np.unique(y_train))).cuda()

# --- Training setup ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-2)

num_epochs = 100
total_steps = len(train_loader) * num_epochs
scheduler = linear_schedule_with_warmup(optimizer, num_warmup_steps=total_steps // 10, num_training_steps=total_steps)

best_loss = float('inf')
early_stop_counter = 0

train_losses = []
test_losses = []
val_accuracies = []

print("\n🚀 Training MLP model...")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
    avg_loss_train = running_loss / len(train_loader)

    # --- Evaluation ---
    model.eval()
    running_loss_test = 0.0
    with torch.no_grad():
        outputs = model(X_test_tensor.cuda())
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        acc = accuracy_score(y_test, preds)

        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
            out = model(X_batch)
            loss = criterion(out, y_batch)
            running_loss_test += loss.item()
        avg_loss_test = running_loss_test / len(test_loader)

        train_losses.append(avg_loss_train)
        test_losses.append(avg_loss_test)
        val_accuracies.append(acc)

        print(f"📉 Train Loss: {avg_loss_train:.4f} | 🧪 Test Loss: {avg_loss_test:.4f} | 🎯 Val Accuracy: {acc:.4f}")

    if avg_loss_test < best_loss:
        best_loss = avg_loss_test
        torch.save(model.state_dict(), "mlp_classifier_best.pt")
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= 10:
            print("🛑 Early stopping triggered (based on test loss).")
            break

# --- Plot and save training/test loss ---
plt.figure()
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()
plt.savefig("loss_plot.png")
print("📊 Saved loss plot to loss_plot.png")

# --- Plot and save validation accuracy ---
plt.figure()
plt.plot(val_accuracies, label='Validation Accuracy', color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy over Epochs')
plt.legend()
plt.savefig("accuracy_plot.png")
print("📈 Saved accuracy plot to accuracy_plot.png")

# --- Final evaluation ---
print("\n🔍 Evaluating best model...")
model.load_state_dict(torch.load("mlp_classifier_best.pt"))
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor.cuda())
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    acc = accuracy_score(y_test, preds)
    print(f"\n✅ Accuracy: {acc:.4f}")
    print("\n📊 Classification Report:")
    print(classification_report(y_test, preds, digits=4))

# --- Save classification report to CSV ---
report_dict = classification_report(y_test, preds, output_dict=True)
df_report = pd.DataFrame(report_dict).transpose()

df_report = df_report[df_report.index.to_series().str.isdigit()]
df_report["label_index"] = df_report.index.astype(int)
df_report["identity"] = df_report["label_index"].map(idx_to_label)
df_report["underrepresented"] = df_report["support"] < 5
df_report["not_predicted"] = df_report["recall"] == 0.0

df_report = df_report[[
    "identity", "label_index", "support", "precision", "recall", "f1-score",
    "underrepresented", "not_predicted"
]]

df_report.to_csv("class_wise_metrics.csv", index=False)
print("📁 Saved class-wise classification report to class_wise_metrics.csv")

# --- Query inference ---
print("\n🔍 Running inference on query set...")

# Load query embeddings
query_embeddings = []
query_ids = []

for k in keys:
    data = np.load(f"query_embeddings_{k}.npz")
    X_query_k = pool_if_needed(data["X"])
    query_embeddings.append(X_query_k)
    if len(query_ids) == 0:
        query_ids = data["image_ids"]

X_query = np.concatenate(query_embeddings, axis=1)

# Apply scaler and PCA
scaler = joblib.load("mlp_scaler.joblib")
pca = joblib.load("mlp_pca.joblib")

X_query_scaled = scaler.transform(X_query)
X_query_pca = pca.transform(X_query_scaled)

# Predict with MLP
X_query_tensor = torch.tensor(X_query_pca, dtype=torch.float32).cuda()
model.eval()
with torch.no_grad():
    outputs = model(X_query_tensor)
    probs = torch.softmax(outputs, dim=1).cpu().numpy()
    preds = np.argmax(probs, axis=1)

# Map predicted class indices back to labels
pred_labels = [idx_to_label[p] for p in preds]
confidences = probs.max(axis=1)

# Save predictions
df_out = pd.DataFrame({
    "image_id": query_ids,
    "identity": pred_labels,
    "confidence": confidences
})
df_out.to_csv("query_predictions.csv", index=False)
print(f"📁 Saved query predictions to query_predictions.csv")

Writing mlp_classifier.py


# **Mega-Descriptor**

In [ ]:
%%writefile train_mega.py
import os
import numpy as np
import pandas as pd
import timm
import torchvision.transforms as T
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.similarity.wildfusion import SimilarityPipeline, WildFusion
from wildlife_tools.similarity.pairwise.lightglue import MatchLightGlue
from wildlife_tools.features.local import AlikedExtractor
from wildlife_tools.similarity.calibration import IsotonicCalibration
from torch.utils.data import Dataset
from PIL import Image
from tqdm import tqdm
import torch

class InferenceDataset(Dataset):
    def __init__(self, df, root_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.metadata = self.df
        self.col_label = "identity"
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.df.iloc[idx]['path'])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, idx

# Paths
root = './animal-clef-2025'

# Transforms
transform_display = T.Compose([T.Resize([384, 384])])
transform = T.Compose([
    *transform_display.transforms,
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
transforms_aliked = T.Compose([
    T.Resize([512, 512]),
    T.ToTensor()
])

# Load metadata
df_database = pd.read_csv("database_metadata.csv")
df_query = pd.read_csv("query_metadata.csv")
df_calibration = pd.read_csv("test_database_metadata.csv")

n_query = len(df_query)

# Datasets
calib_ds = InferenceDataset(df_calibration, root, transform)
query_ds = InferenceDataset(df_query, root, transform)
database_ds = InferenceDataset(df_database, root, transform)

# Load MegaDescriptor model
print("🔄 Loading MegaDescriptor model...")
model = timm.create_model('hf-hub:BVRA/MegaDescriptor-L-384', num_classes=0, pretrained=True)
device = 'cuda'

# Define similarity pipelines
matcher_aliked = SimilarityPipeline(
    matcher=MatchLightGlue(features='aliked', device=device, batch_size=16),
    extractor=AlikedExtractor(),
    transform=transforms_aliked,
    calibration=IsotonicCalibration()
)

matcher_mega = SimilarityPipeline(
    matcher=CosineSimilarity(),
    extractor=DeepFeatures(model=model, device=device, batch_size=16),
    transform=transform,
    calibration=IsotonicCalibration()
)

# Calibrate WildFusion with progress bar
print("🧪 Calibrating WildFusion...")
wildfusion = WildFusion(
    calibrated_pipelines=[matcher_aliked, matcher_mega],
    priority_pipeline=matcher_mega
)

with tqdm(total=1, desc="Fitting Calibration") as pbar:
    wildfusion.fit_calibration(calib_ds, calib_ds)
    pbar.update(1)

# Compute similarities
print("🔍 Computing similarities using WildFusion...")
similarity = wildfusion(query_ds, database_ds, B=None)

# Get predictions
pred_idx = similarity.argmax(axis=1)
pred_scores = similarity[np.arange(n_query), pred_idx]
identities = df_database["identity"].tolist()
predictions = [identities[i] for i in pred_idx]
image_ids = df_query["image_id"].tolist()

# Create submission
submission = pd.DataFrame({
    "image_id": image_ids,
    "identity": predictions,
    "confidence": pred_scores
})

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv has been created successfully.")

# Save the model and WildFusion object
save_dict = {
    'model_state_dict': model.state_dict(),
    'wildfusion': wildfusion,
    'df_database': df_database,
    'df_query': df_query,
    'df_calibration': df_calibration
}

torch.save(save_dict, "wildfusion_model.pth")
print("💾 Model saved to 'wildfusion_model.pth'")

Writing train_mega.py


# **Model Soup**

In [ ]:
%%writefile model_soup.py
import os
import csv
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import get_scheduler
import torchvision.transforms as T
import timm

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset class
class AnimalDataset(Dataset):
    def __init__(self, csv_file, root_dir, label_encoder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['path'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label_str = self.data.iloc[idx]['identity']
        label = self.label_encoder[label_str]
        return image, label

# Early stopping
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0

    def should_stop(self, current_loss):
        if current_loss < self.best_loss - self.min_delta:
            self.best_loss = current_loss
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Train and validation functions
def train_one_epoch(model, dataloader, optimizer, scheduler, criterion, device, epoch):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(dataloader, desc=f"Training (Epoch {epoch+1})"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

def validate(model, dataloader, criterion, device, epoch=0):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc=f"Validating (Epoch {epoch+1})"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    return running_loss / len(dataloader), accuracy

# Load model state dict
def load_state_dict(path):
    checkpoint = torch.load(path, map_location='cpu')
    return checkpoint['model_state_dict']

# Average multiple model weights
def weighted_average_models(model_paths, weights, model_arch, num_classes, device):
    assert len(model_paths) == len(weights), "Numărul de modele și de ponderi trebuie să coincidă."
    weights = [float(w) for w in weights]
    total_weight = sum(weights)
    norm_weights = [w / total_weight for w in weights]

    base_model = timm.create_model(model_arch, pretrained=False, num_classes=num_classes).to(device)
    avg_state_dict = None

    for i, path in enumerate(model_paths):
        state_dict = load_state_dict(path)
        weight = norm_weights[i]
        if avg_state_dict is None:
            avg_state_dict = {k: v.clone().float() * weight for k, v in state_dict.items()}
        else:
            for k in avg_state_dict:
                avg_state_dict[k] += state_dict[k].float() * weight

    base_model.load_state_dict(avg_state_dict)
    return base_model

# Paths
root = './animal-clef-2025'
train_csv = "train_database_metadata.csv"
test_csv = "test_database_metadata.csv"

# Transforms
train_transform = T.Compose([
    T.RandomResizedCrop(384, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ColorJitter(0.4, 0.4, 0.4, 0.1),
    T.AutoAugment(policy=T.AutoAugmentPolicy.IMAGENET),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    T.RandomErasing(p=0.25)
])
test_transform = T.Compose([
    T.Resize([384, 384]),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

# Label encoder
train_identities = pd.read_csv(train_csv)['identity'].unique()
label_encoder = {identity: idx for idx, identity in enumerate(sorted(train_identities))}

# Dataloaders
batch_size = 8
train_dataset = AnimalDataset(train_csv, root, label_encoder, train_transform)
test_dataset = AnimalDataset(test_csv, root, label_encoder, test_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Train a single model (optional training loop - can skip if you're only testing soup)
model = timm.create_model('convnext_xlarge.fb_in22k_ft_in1k_384', pretrained=True, num_classes=len(label_encoder)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=8e-5, weight_decay=0.01)
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=int(0.1 * len(train_loader) * 30), num_training_steps=len(train_loader) * 30)

# -------------------------------
# Evaluate Weighted Model Soup
# -------------------------------
model_paths = ["best_model_1.pth", "best_model_2.pth", "best_model_3.pth"]
weights = [0.7, 0.15, 0.15]

model_soup = weighted_average_models(model_paths, weights, 'convnext_xlarge.fb_in22k_ft_in1k_384', len(label_encoder), device)
model_soup.eval()

# Save the weighted soup model
soup_model_path = "weighted_model_soup.pth"
torch.save({
    'model_state_dict': model_soup.state_dict(),
    'label_encoder': label_encoder
}, soup_model_path)
print(f"💾 Weighted Model Soup saved to {soup_model_path}")

test_loss, test_acc = validate(model_soup, test_loader, criterion, device)
print(f"📊 Weighted Model Soup Evaluation -> Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

Writing model_soup.py


# **Soft Ensemble Evaluation**

In [ ]:
%%writefile model_soup.py
import os
import csv
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import get_scheduler
import torchvision.transforms as T
import timm
import torch.nn.functional as F

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset class
class AnimalDataset(Dataset):
    def __init__(self, csv_file, root_dir, label_encoder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['path'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label_str = self.data.iloc[idx]['identity']
        label = self.label_encoder[label_str]
        return image, label

# Early stopping
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.0005):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0

    def should_stop(self, current_loss):
        if current_loss < self.best_loss - self.min_delta:
            self.best_loss = current_loss
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Train and validation functions
def train_one_epoch(model, dataloader, optimizer, scheduler, criterion, device, epoch):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(dataloader, desc=f"Training (Epoch {epoch+1})"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

def validate(model, dataloader, criterion, device, epoch=0):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc=f"Validating (Epoch {epoch+1})"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    return running_loss / len(dataloader), accuracy

# Load model state dict
def load_state_dict(path):
    checkpoint = torch.load(path, map_location='cpu')
    return checkpoint['model_state_dict']

def evaluate_soft_ensemble(model1, model2, dataloader, criterion, device, alpha=0.5):
    model1.eval()
    model2.eval()

    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="🧪 Evaluating Soft Ensemble"):
            images = images.to(device)
            labels = labels.to(device)

            # Predict logits
            logits1 = model1(images)
            logits2 = model2(images)

            # Convert to softmax probabilities
            probs1 = F.softmax(logits1, dim=1)
            probs2 = F.softmax(logits2, dim=1)

            # Weighted average
            blended_probs = alpha * probs1 + (1 - alpha) * probs2

            # Compute loss
            loss = criterion(blended_probs.log(), labels)
            total_loss += loss.item()

            # Final prediction
            preds = blended_probs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    return avg_loss, accuracy

# Paths
root = './animal-clef-2025'
train_csv = "train_database_metadata.csv"
test_csv = "test_database_metadata.csv"

# Transforms
train_transform = T.Compose([
    T.RandomResizedCrop(384, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ColorJitter(0.4, 0.4, 0.4, 0.1),
    T.AutoAugment(policy=T.AutoAugmentPolicy.IMAGENET),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    T.RandomErasing(p=0.25)
])
test_transform = T.Compose([
    T.Resize([384, 384]),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

# Label encoder
train_identities = pd.read_csv(train_csv)['identity'].unique()
label_encoder = {identity: idx for idx, identity in enumerate(sorted(train_identities))}

# Dataloaders
batch_size = 8
train_dataset = AnimalDataset(train_csv, root, label_encoder, train_transform)
test_dataset = AnimalDataset(test_csv, root, label_encoder, test_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Load both models
model1 = timm.create_model('convnext_xlarge.fb_in22k_ft_in1k_384', pretrained=False, num_classes=len(label_encoder)).to(device)
model1.load_state_dict(torch.load("best_model_convnext.pth", map_location=device)['model_state_dict'])

model2 = timm.create_model('maxvit_xlarge_tf_384.in21k_ft_in1k', pretrained=False, num_classes=len(label_encoder)).to(device)
model2.load_state_dict(torch.load("best_model_maxvit.pth", map_location=device)['model_state_dict'])

# Evaluate
alpha = 0.6  # weight for model1
soft_ensemble_loss, soft_ensemble_acc = evaluate_soft_ensemble(model1, model2, test_loader, criterion, device, alpha)
print(f"📊 Soft Ensemble (α={alpha}) -> Test Loss: {soft_ensemble_loss:.4f} | Test Accuracy: {soft_ensemble_acc:.4f}")

# **Inference**

In [ ]:
%%writefile inference_query.py
import torch
import timm
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
from tqdm import tqdm

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = "best_model.pth"
checkpoint = torch.load(model_path, map_location=device)

label_encoder = checkpoint['label_encoder']
idx_to_label = {v: k for k, v in label_encoder.items()}

model = timm.create_model(
    'maxvit_xlarge_tf_384.in21k_ft_in1k',
    pretrained=False,
    num_classes=len(label_encoder)
)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval()

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

# Prediction function without threshold filtering
def predict(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        probs = torch.softmax(outputs, dim=1)
        conf, preds = probs.max(dim=1)
        confidence = conf.item()
        predicted_idx = preds.item()

    predicted_label = idx_to_label[predicted_idx]
    return predicted_label, confidence

# Load query data
query_csv = "query_metadata.csv"
query_data = pd.read_csv(query_csv)

root = "./animal-clef-2025"
predictions = []
confidences = []

# Inference loop
for idx, row in tqdm(query_data.iterrows(), total=len(query_data), desc="Predicting"):
    img_rel_path = row['path']
    img_full_path = os.path.join(root, img_rel_path)

    try:
        pred_label, conf = predict(img_full_path)
    except Exception as e:
        print(f"❌ Error with image {img_full_path}: {e}")
        pred_label, conf = "error", 0.0

    predictions.append(pred_label)
    confidences.append(conf)

# Save results
query_data['identity'] = predictions
query_data['confidence'] = confidences

output_csv = "query_with_predictions.csv"
query_data.to_csv(output_csv, index=False)

print(f"✅ File saved at {output_csv}")

Writing inference_query.py



# **Fusion Strategy**

In [ ]:
import pandas as pd

# Load the two prediction files
df1 = pd.read_csv("preds_1_mega_descriptor.csv")
df2 = pd.read_csv("preds_4_mega_descriptor.csv")

# Rename columns to avoid name conflicts
df1 = df1.rename(columns={"identity": "identity_1", "confidence": "confidence_1"})
df2 = df2.rename(columns={"identity": "identity_2", "confidence": "confidence_2"})

# Merge the dataframes on 'image_id'
merged = df1.merge(df2, on="image_id")

# Prepare the final results
result_rows = []

for _, row in merged.iterrows():
    image_id = row["image_id"]

    identity_1 = row["identity_1"]
    identity_2 = row["identity_2"]

    # Weighted confidence: slightly favor the second model
    confidence_1 = row["confidence_1"] * 0.45
    confidence_2 = row["confidence_2"] * 0.55

    if identity_1 == identity_2:
        final_identity = identity_1
        final_confidence = (confidence_1 + confidence_2) * 1.2
    else:
        final_identity = identity_2
        final_confidence = confidence_2 * 1.82

    result_rows.append({
        "image_id": image_id,
        "identity": final_identity,
        "confidence": final_confidence
    })

# Create and save the final DataFrame
result_df = pd.DataFrame(result_rows)
result_df.to_csv("preds_mega_descriptor.csv", index=False)

print("✅ File 'preds_mega_descriptor.csv' has been created successfully.")

✅ File 'preds_mega_descriptor.csv' has been created successfully.


In [ ]:
import pandas as pd

# Load predictions from the two models
df_mega = pd.read_csv("preds_mega_descriptor.csv")
df_conv1 = pd.read_csv("preds_convnext_1.csv")

# Rename columns to avoid conflicts
df_mega = df_mega.rename(columns={"identity": "identity_mega", "confidence": "confidence_mega"})
df_conv1 = df_conv1.rename(columns={"identity": "identity_conv1", "confidence": "confidence_conv1"})

# Merge predictions on image_id
merged = df_mega.merge(df_conv1, on="image_id")

# Prepare final results
result_rows = []

for _, row in merged.iterrows():
    image_id = row["image_id"]

    identity_mega = row["identity_mega"]
    identity_conv1 = row["identity_conv1"]

    confidence_mega = row["confidence_mega"]
    confidence_conv1 = row["confidence_conv1"] * 0.25

    if identity_mega == identity_conv1:
        final_identity = identity_mega
        final_confidence = ((confidence_mega + confidence_conv1) / 2) * 1.7
    else:
        if confidence_mega >= confidence_conv1:
            final_identity = identity_mega
            final_confidence = confidence_mega
        else:
            final_identity = identity_conv1
            final_confidence = confidence_conv1 * 1.2

    result_rows.append({
        "image_id": image_id,
        "identity": final_identity,
        "confidence": final_confidence
    })

# Create and save final DataFrame
result_df = pd.DataFrame(result_rows)
result_df.to_csv("final_submission.csv", index=False)

print("✅ The file 'final_submission.csv' has been created using df_mega and df_conv1.")


✅ The file 'final_submission.csv' has been created using df_mega and df_conv1.


In [ ]:
import pandas as pd

# Load the CSV
df = pd.read_csv("final_submission.csv")

# Scale confidence values to percentage
df["confidence_percent"] = df["confidence"] * 100

# Define bins and labels
bins = [30, 35, 40, 45, 50, 52, 54, 55, 60, 65, 70, 75, 80, 85]
labels = ["30–35", "35–40", "40–45", "45–50", "50-52", "52-54", "54-55", "55–60", "60–65", "65–70", "70–75", "75–80", "80–85"]

# Bin the data
df["interval"] = pd.cut(df["confidence_percent"], bins=bins, labels=labels, right=False)

# Count occurrences per interval
counts = df["interval"].value_counts().sort_index()

print("Confidence (%) value distribution per interval:")
print(counts)

Confidence (%) value distribution per interval:
interval
30–35    146
35–40    178
40–45    289
45–50     82
50-52     41
52-54     21
54-55     15
55–60     35
60–65     60
65–70     77
70–75     45
75–80     36
80–85     35
Name: count, dtype: int64


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("final_submission.csv")

# Apply threshold: if confidence < 0.6, set identity to 'new_individual'
df.loc[df["confidence"] < 0.45, "identity"] = "new_individual"

# Save the updated DataFrame back to CSV
df.to_csv("final_submission_thresholded.csv", index=False)

print("The file 'final_submission_thresholded.csv' has been created with the threshold applied.")

The file 'final_submission_thresholded.csv' has been created with the threshold applied.


# **Sample Submission**

In [ ]:
import pandas as pd

query_predictions = pd.read_csv('final_submission_thresholded.csv')

submission = query_predictions[['image_id', 'identity']]

submission.to_csv('sample_submission.csv', index=False)

print("✅ sample_submission.csv done!")

✅ sample_submission.csv done!
